In [1]:
from scapy.all import *
import pandas as pd
import numpy as np
import binascii
import seaborn as sns

In [2]:
applicationData = rdpcap("D:\\BotnetDetection\\isot_app_and_botnet_dataset\\application_data\\applicationDNS.pcap")

In [3]:
applicationData

<applicationDNS.pcap: TCP:0 UDP:652 ICMP:0 Other:430>

In [4]:
ethernet_frame = applicationData[23]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload
print(ethernet_frame.summary())
print(ip_packet.summary())
print(segment.summary())
print(data.summary())
ethernet_frame.show()

Ether / IP / UDP / DNS Qry "b'clients2.google.com.'" 
IP / UDP / DNS Qry "b'clients2.google.com.'" 
UDP / DNS Qry "b'clients2.google.com.'" 
DNS Qry "b'clients2.google.com.'" 
###[ Ethernet ]### 
  dst       = 08:00:27:e8:fe:87
  src       = 08:00:27:96:32:51
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 65
     id        = 1938
     flags     = 
     frag      = 0
     ttl       = 128
     proto     = udp
     chksum    = 0x4d3f
     src       = 192.168.50.50
     dst       = 192.168.50.88
     \options   \
###[ UDP ]### 
        sport     = 55066
        dport     = domain
        len       = 45
        chksum    = 0xc381
###[ DNS ]### 
           id        = 17701
           qr        = 0
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 1
           ra        = 0
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
      

In [5]:
applicationData[23].pdfdump(layer_shift=1)

In [6]:
ethernet_type = type(ethernet_frame)
ip_type = type(ip_packet)
tcp_type = type(segment)
print("Ethernet",applicationData[ethernet_type])
print("IP", applicationData[ip_type])
print("UDP", applicationData[tcp_type])

Ethernet <Ether from applicationDNS.pcap: TCP:0 UDP:652 ICMP:0 Other:430>
IP <IP from applicationDNS.pcap: TCP:0 UDP:590 ICMP:0 Other:0>
UDP <UDP from applicationDNS.pcap: TCP:0 UDP:652 ICMP:0 Other:0>


In [7]:
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]
dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']
df = pd.DataFrame(columns=dataframe_fields)
for packet in applicationData[IP]:
    field_values = []
    for field in ip_fields:
        if field == 'options':
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    field_values.append(packet.time)
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)
df = df.reset_index()
df = df.drop(columns="index")

In [8]:
df.head()
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,192.168.50.88,192.168.50.51,53,61485
1,192.168.50.19,192.168.50.88,60019,53
2,192.168.50.88,8.8.4.4,51316,53
3,192.168.50.51,192.168.50.88,55221,53
4,192.168.50.51,192.168.50.88,59936,53
...,...,...,...,...
585,192.168.50.51,192.168.50.255,137,137
586,192.168.50.19,192.168.50.88,62476,53
587,192.168.50.88,8.8.4.4,55706,53
588,192.168.50.51,192.168.50.255,137,137
